# Лабораторная работа 4 : Задача восстановления зависимостей. Линейная регрессия. Бустинг.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold


## Задание 1

In [ ]:
# V = 83 => вариант 3

df = pd.read_csv("avocado.csv", parse_dates=True)
df = df.sample(frac=1)
df = df.reset_index()
df = df.rename(columns={"4046": "Small Hass",
               "4225": "Large Hass", "4770": "XLarge Hass"})
df = df.drop(columns=["Unnamed: 0", "index"])

stringValues = ['Date', 'type', 'region']
df[stringValues] = df[stringValues].apply(lambda x: pd.factorize(x)[0])

df


In [ ]:
df.info()


In [ ]:
print('статистики')
df.describe()


In [ ]:
print('пропущенных значений', df.apply(
    lambda x: sum(x.isnull()), axis=0).sum())


In [ ]:
print('Матрица корреляций:')
sns.heatmap(df.corr(), cmap='viridis', annot=True)


In [ ]:
for c in df.columns:
    print(c)
    print(df[c].describe())
    sns.boxplot(y=c, data=df)
    plt.show()


Видно, что чистка данных не требуется.

In [ ]:
X = df.dropna().reset_index().drop('AveragePrice', axis=1)
y = df.dropna().reset_index()['AveragePrice']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=838383)


## Задание 2

In [ ]:
LR = LinearRegression()
LR.fit(X_train, y_train)
Y_LR = LR.predict(X_test)
print('Mean absolute error:', round(mean_absolute_error(y_test, Y_LR), 3))
print('Sqrt of mean squared error:', round(mean_squared_error(y_test, Y_LR)**(1/2), 3))
print('R2 score:', round(r2_score(y_test, Y_LR), 3))


In [ ]:
coefs = abs(LR.coef_)
feature_importance = pd.DataFrame(
    columns=['Название признака', 'Важность признака'])
feature_importance['Название признака'] = X.keys()
feature_importance['Важность признака'] = coefs/sum(coefs)
feature_importance = feature_importance.sort_values(
    by='Важность признака', ascending=False)
print('Наиболее информативные признаки')
feature_importance


Наиболее информативный - тип

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(y_test, Y_LR)
plt.plot([0, 3.3], [0, 3.3], color='g')
plt.xlabel('Real AveragePrice', fontsize=20)
plt.ylabel('Predict AveragePrice', fontsize=20)


## Задание 3

In [ ]:
gb = XGBRegressor(objective='reg:squarederror')
gb.fit(X_train, y_train)
Y_GB = gb.predict(X_test)
print('Mean absolute error:', round(mean_absolute_error(y_test, Y_GB), 3))
print('Sqrt of mean squared error:', round(mean_squared_error(y_test, Y_GB)**(1/2), 3))
print('R2 score:', round(r2_score(y_test, Y_GB), 3))


Видно, что R2 бустингом получился в 2 раза больше, чем лин. регрессией.

In [ ]:
feature_importance = pd.DataFrame(
    columns=['Название признака', 'Важность признака'])
feature_importance['Название признака'] = X.keys()
feature_importance['Важность признака'] = gb.feature_importances_
feature_importance = feature_importance.sort_values(
    by='Важность признака', ascending=False)
print('Наиболее информативные признаки')
feature_importance


Еаиболее информативный также - тип

In [ ]:
plt.figure(figsize=(7, 7))
plt.scatter(y_test, Y_GB)
plt.plot([0, 3.3], [0, 3.3], color='g')
plt.xlabel('Real AveragePrice', fontsize=20)
plt.ylabel('Predict AveragePrice', fontsize=20)


In [ ]:
n = 4
cv_model_lr = cross_validate(LR, X, y, cv=RepeatedKFold(
    n_splits=n, n_repeats=n), return_estimator=False, n_jobs=1)
cv_model_gb = cross_validate(gb, X, y, cv=RepeatedKFold(
    n_splits=n, n_repeats=n), return_estimator=False, n_jobs=1)
x_i = range(0, n*n)
y_1 = cv_model_lr['test_score']
y_2 = cv_model_gb['test_score']
plt.subplots(figsize=(7, 7))
plt.title('CV_Score')
plt.plot(x_i, y_1, label='LR')
plt.plot([0, n*n - 1], [y_1.mean()]*2, label='mean LR')
plt.plot(x_i, y_2, label='GB')
plt.plot([0, n*n - 1], [y_2.mean()]*2, label='mean GB')
plt.legend()


## Задание 4

Помним, что наиболее информативная - тип

In [ ]:
start = 0
end = 15
plt.subplots(figsize=(end, end//2))
plt.plot(X.index[start:end], y[start:end], label='Real values')
plt.scatter((X[start:end][X['type'] == 0]).index,
            y[start:end][X['type'] == 0], label='type 0', c='y')
plt.scatter((X[start:end][X['type'] == 1]).index,
            y[start:end][X['type'] == 1], label='type 1', c='r')
plt.plot(X.index[start:end], LR.predict(X)[start:end], label='LR')
plt.plot(X.index[start:end], gb.predict(X)[start:end], label='GB')
plt.legend()
